In [8]:
import os
import pandas as pd
from pandas.plotting import table 
import numpy as np
import typing as tp

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.preprocessing import LabelEncoder

from scipy.stats.stats import kendalltau

import dataframe_image as dfi
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings 
warnings.filterwarnings('ignore')

Let's upload the data with the results and build graphs showing the quality of the models.

In [9]:
path_to_data = './input_data/results_with_features_relative_to_NaiveForecasterSktime.xlsx'
path_to_plots = './results'

dataset_name = 'M5'
hor = 7
metrics = ['MAE', 'MSE', 'RMSE', 'MASE', 'RMSSE', 'SMAPE']
sort_metric = 'SMAPE'
N = 20

In [10]:
def load_prep_data(path_to_data):
    '''
    A function for uploading data
    '''
    res = pd.read_excel(f'{path_to_data}')
    res = res.drop(columns = ['Unnamed: 0'])
    res = res.rename(columns = {'model': 'model_name', 'dataset': 'dataset_path'})
    ts_lst = list(res['naming_orig'].value_counts().index)
    return res
    
    

In [11]:
def plot_metrics_table(res, path_to_plots, dataset_name, hor, metrics, sort_metric, N):
    '''
    A function for plotting graphs
    res - DataFrame with results of experiments (model metrics)
    path_to_plots - the path to save the results
    dataset_name - name of the dataset or datasets for plotting
    hor - horizon of predictions
    metrics - metrics to show on the graph
    sort_metric - main metrics for sorting
    N - number of top models
    '''
    
    res_grouped = res[(res['dataset_path'] == dataset_name)&(res['horizon'] == hor)].groupby(['model_name'])[metrics].agg({'MAE':'mean', 'MSE':'mean', 'RMSE': 'mean', 'MASE':'mean', 'RMSSE':'mean', 'SMAPE': ['mean', 'std']})
    pred_time_top_all = res_grouped.sort_values(by = [(sort_metric, 'mean')], ascending = True).rename(columns={"pred_time": "train_time"}).head(N)
    best_N_m = list(pred_time_top_all.head(N).index)
    display(pred_time_top_all)
    df_styled_all = pred_time_top_all.style.background_gradient().format('{:.3f}')
    dfi.export(df_styled_all, path_to_plots + f'results_std_{hor}_' + dataset_name + '.png', fontsize=4, dpi=1000)
    
    return best_N_m

In [ ]:
N = 20
sort_metric = 'SMAPE'
lst_datasets = ['M5', 'favorita_stores_forecasting', 'demand_forecasting_kernels', 'pems04', 'pems08', 'synth_S-AL', 'synth_S-EF', 'synth_S-TSAL', 'synth_S-TSEFAL', 'synth_S-TSEF', 'synth_S-TS', 'synth_S-mTSEF', 'traffic_junctions'] 

res = load_prep_data(path_to_data)

for i in lst_datasets:
    for hor in [7, 30, 90, 365]:
        plot_metrics_table(res, path_to_plots, i, hor, metrics, sort_metric, N)

In [8]:
df = load_prep_data(path_to_data)
df.head()

,naming_orig,horizon,model_name,MAE,MSE,RMSE,MASE,RMSSE,SMAPE,dataset_path,...,value__quantile__q_0.75,value__number_peaks__n_1,value__number_peaks__n_3,value__number_peaks__n_5,value__number_peaks__n_10,value__value_count__value_0,"value__augmented_dickey_fuller__attr_""pvalue""__autolag_""t-stats""","value__augmented_dickey_fuller__attr_""teststat""__autolag_""t-stats""",dataset_name,length
0,M5_sample_1,7,ARIMAEtna,1.018917,0.975314,0.987580,1.018917,0.987580,0.992877,M5,...,0.0,80,34,20,10,1792,NaN,NaN,M5_sample_1_TX_HOUSEHOLD.csv,1941
1,M5_sample_1,30,ARIMAEtna,1.454314,0.947985,0.973645,1.454314,0.973645,0.995468,M5,...,0.0,80,34,20,10,1792,NaN,NaN,M5_sample_1_TX_HOUSEHOLD.csv,1941
2,M5_sample_1,90,ARIMAEtna,23.260741,12.036061,3.469303,23.260741,3.469303,0.984611,M5,...,0.0,80,34,20,10,1792,NaN,NaN,M5_sample_1_TX_HOUSEHOLD.csv,1941
3,M5_sample_1,365,ARIMAEtna,1.689799,0.864922,0.930012,1.689799,0.930012,0.912725,M5,...,0.0,80,34,20,10,1792,NaN,NaN,M5_sample_1_TX_HOUSEHOLD.csv,1941
4,M5_sample_1,7,ARIMAKats,1.018557,0.975750,0.987801,1.018557,0.987801,0.993005,M5,...,0.0,80,34,20,10,1792,NaN,NaN,M5_sample_1_TX_HOUSEHOLD.csv,1941


In [9]:
df.columns

Index(['naming_orig', 'horizon', 'model_name', 'MAE', 'MSE', 'RMSE', 'MASE',
       'RMSSE', 'SMAPE', 'dataset_path', 'value__binned_entropy__max_bins_50',
       'value__maximum', 'value__minimum', 'value__sample_entropy',
       'value__longest_strike_below_mean', 'value__longest_strike_above_mean',
       'value__mean_abs_change', 'value__has_duplicate', 'value__sum_values',
       'value__mean_change', 'value__median', 'value__mean', 'value__length',
       'value__standard_deviation', 'value__variance', 'value__skewness',
       'value__kurtosis',
       'value__percentage_of_reoccurring_values_to_all_values',
       'value__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'value__linear_trend__attr_"pvalue"',
       'value__linear_trend__attr_"slope"',
       'value__linear_trend__attr_"intercept"',
       'value__ar_coefficient__coeff_0__k_10',
       'value__ar_coefficient__coeff_1__k_10',
       'value__ar_coefficient__coeff_2__k_10',
       'value__partial_autocorrelatio

# Ranking model

In [9]:
!pip install xgboost

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import ndcg_score
from scipy.stats.stats import kendalltau

import seaborn as sns
sns.set()
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_excel(f'{path_to_data}')

Features for building a ranking model

In [ ]:
features_names = ['model_name', 'value__binned_entropy__max_bins_50', 'value__maximum',
       'value__minimum', 'value__sample_entropy',
       'value__longest_strike_below_mean', 'value__longest_strike_above_mean',
       'value__mean_abs_change', 'value__has_duplicate', 'value__sum_values',
       'value__mean_change', 'value__median', 'value__mean', 'value__length',
       'value__standard_deviation', 'value__variance', 'value__skewness',
       'value__kurtosis',
       'value__percentage_of_reoccurring_values_to_all_values',
       'value__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'value__linear_trend__attr_"pvalue"',
       'value__linear_trend__attr_"slope"',
       'value__linear_trend__attr_"intercept"',
       'value__ar_coefficient__coeff_0__k_10',
       'value__ar_coefficient__coeff_1__k_10',
       'value__ar_coefficient__coeff_2__k_10',
       'value__partial_autocorrelation__lag_0',
       'value__partial_autocorrelation__lag_1',
       'value__partial_autocorrelation__lag_2',
       'value__symmetry_looking__r_0.1', 'value__symmetry_looking__r_0.5',
       'value__quantile__q_0.25', 'value__quantile__q_0.75',
       'value__number_peaks__n_1', 'value__number_peaks__n_3',
       'value__number_peaks__n_5', 'value__number_peaks__n_10',
       'value__value_count__value_0',
       'value__augmented_dickey_fuller__attr_"pvalue"__autolag_"t-stats"',
       'value__augmented_dickey_fuller__attr_"teststat"__autolag_"t-stats"',
       'length']

A function for dividing time series into training and test parts. The number of time series in the training sample in the example below is 67 % of the total number of time series. The division of time series occurred randomly.

In [ ]:
seed = 7
test_size = 0.33

res = df.copy()

lst_ts = list(set(res['naming_orig']))
ts_train, ts_test = train_test_split(lst_ts, test_size=test_size, random_state=seed)

mask_train = []
mask_test = []
for i in range(len(res['naming_orig'])):
    if res['naming_orig'][i] in ts_train:
        mask_train.append(True)
        mask_test.append(False)
    else:
        mask_train.append(False)
        mask_test.append(True)
        
data_train = res[mask_train]
data_test = res[mask_test]

data_train.to_csv('./input_data/ts_train.csv', index = False)
data_test.to_csv('./input_data/ts_test.csv', index = False)

Сonsider the forecasting horizon 7. Firstly, selecting data for the appropriate horizon.

In [ ]:
data_train = pd.read_csv('./input_data/ts_train.csv')
data_test = pd.read_csv('./input_data/ts_test.csv')

data_train_7 = data_train[data_train['horizon'] == 7]
data_test_7 = data_test[data_test['horizon'] == 7]

In [ ]:
def train_Rank_model(hor, features_names):
    data_train_hor = data_train[data_train['horizon'] == hor]
    data_test_hor = data_test[data_test['horizon'] == hor]
    data_train_hor['rank'] = data_train_hor.sort_values(['SMAPE'], ascending=True).groupby(['naming_orig'])['SMAPE'].cumcount() + 1
    data_test_hor['rank'] = data_test_hor.sort_values(['SMAPE'], ascending=True).groupby(['naming_orig'])['SMAPE'].cumcount() + 1
      
    # features and target elimination
    X_train_hor = data_train_hor.drop(columns = ['rank', 'MAE', 'MSE', 'RMSE', 'MASE', 'RMSSE', 'SMAPE'])
    y_train_hor = data_train_hor['rank']
    X_test_hor = data_test_hor.drop(columns = ['rank', 'MAE', 'MSE', 'RMSE', 'MASE', 'RMSSE', 'SMAPE'])
    y_test_hor = data_test_hor['rank']
    
    le = LabelEncoder()
    label_encoded_X_train_hor = X_train_hor.drop(columns = ['naming_orig']).copy()
    label_encoded_X_test_hor = X_test_hor.drop(columns = ['naming_orig']).copy()
    
    for col in label_encoded_X_train_hor[features_names].select_dtypes(include='O').columns:
        label_encoded_X_train_hor[col]=le.fit_transform(label_encoded_X_train_hor[col])
        label_encoded_X_test_hor[col]=le.transform(label_encoded_X_test_hor[col])
        
    # division into groups
    groups = np.array([35]* len(X_train_7['naming_orig'].value_counts().index))
    
    # model fitting
    model = xgb.XGBRanker(max_depth=5, learning_rate=0.01, n_estimators=3000, n_jobs=-1, colsample_bytree=0.1)
    model.fit(label_encoded_X_train_hor[features_names], y_train_hor, group = groups, verbose=True)
    feature_importance = model.feature_importances_
    sorted_idx = np.argsort(feature_importance)
    print(f'Важность признаков (горизонт {hor})')
    plot_importance(model)
    plt.title(f'Важность признаков (горизонт {hor})')
        
        
    # train-test predictions
    y_pred_hor_train = model.predict(label_encoded_X_train_hor[features_names])
    y_pred_hor_test = model.predict(label_encoded_X_test_hor[features_names])
    
    # concatinating of the forecasts
    pred_rank_hor_train = pd.concat([X_train_hor.reset_index(), pd.DataFrame(y_pred_hor_train).reset_index()], axis = 1)
    pred_rank_hor_test = pd.concat([X_test_hor.reset_index(), pd.DataFrame(y_pred_hor_test).reset_index()], axis = 1)
    
    pred_rank_hor_test['rank'] = pred_rank_hor_test.sort_values([0], ascending=True).groupby(['naming_orig'])[0].cumcount() + 1
    display(pred_rank_hor_test.head(1))
    
    SMAPE_mean_test = pred_rank_hor_test[pred_rank_hor_test['rank'] == 1]['SMAPE'].mean()
    print('SMAPE mean test: ', SMAPE_mean_test)
    
    return SMAPE_mean_test

Data preparation stage: sample was divided into train and test part in the ratio 40:60. 

In [ ]:
train_Rank_model(7, features_names)

Example of data splitting for horizon 7.

# Portfolio

In [ ]:
PATH_TO_RESULTS = './results'
METRICS = ['MAE','MSE','SMAPE']
METRIC = 'SMAPE'
DATASETS = ['M5', 'favorita_stores_forecasting', 'demand_forecasting_kernels', 'pems04', 'pems08']

In [ ]:
def GetDataFrameResDataValidata(path: str = PATH_TO_RESULTS,
                                metrics: list[str] = METRICS,
                                dataset: tp.Optional[list[str]] = None) -> tuple[pd.DataFrame, pd.DataFrame]:
    '''
    The function of preparing the source table for further work.
    path - is the path to the pd.Data Frame (scv) data. The data must contain the following fields:
    dataset_name, model_name, split, naming_orig, horizon,
    also, the data for the split (validation, test) part must contain a Prophet
    metrics - a list of metrics that we take into account (you can add everything)
    dataset - a list of the names of the datasets that the portfolio is based on.
    We leave only those rows that are from dataset datasets.
    '''
    all_data = pd.read_csv(path).rename(columns={'dataset_name': 'dataset'})
    all_data.drop_duplicates(subset=['model_name', 'split', 'naming_orig', 'dataset', 'horizon'], inplace=True)
    
    all_data = all_data if dataset is None else all_data.loc[all_data.dataset.apply(lambda x: x in dataset)]
    
    val = all_data.query(f"split == 'validation'")
    test = all_data.query(f"split == 'test'")

    val = val.set_index(['model_name', 'naming_orig']).sort_index()
    val.fillna(val.mean(level=0), inplace=True)
    val.reset_index(inplace=True)
    test = test.set_index(['model_name', 'naming_orig']).sort_index()
    test.fillna(test.mean(level=0), inplace=True)
    test.reset_index(inplace=True)

    return val, test

In [ ]:
def GetPortfolioMatrix(data: pd.DataFrame,
                       horizon: int,
                       metric: list[str] = METRIC,
                       metrics: str = METRICS,
                       ) -> pd.DataFrame:
    '''
    The function of converting the data table to the model/score table on a time series
    data - prepared table of normalized time series results 
           (after function GetDataFrameResDataValidata)
    horizon - the section on which forecasting horizon will be made
    metric - the main metric from the metrics list, according to which the final table for the algorithm will be compiled.
    metrics - the list of metrics that we take into account (you can specify all of them)
    '''
    fillna = data.query("model_name == 'Prophet'").mean(numeric_only=True).loc[metric]
    data_horizon = data \
                    .set_index(['model_name', 'dataset', 'naming_orig']) \
                    [metrics].stack().unstack(level=1).unstack(level=1)

    data_metric = data_horizon.loc[(slice(None), metric), :].sort_index()
    return data_metric.fillna(fillna)

In [ ]:
class GreedyPortfolioConstructionMin:
    def __init__(
      self,
      number: int = 3 
        ):
        self._number = number
        self._matrix = None

    @staticmethod
    def _rebalance(
            matrix: np.array,
            row_number: int
            ) -> np.array:
        currency_row = matrix[row_number]
        rebalance_matrix = np.minimum(matrix, currency_row)
        return rebalance_matrix

    def predict(
        self,
        matrix: np.array
        ) -> list[int]:
        self._matrix = matrix
        portfolio = []
        for step in range(self._number):
            best_row = np.argmin(np.mean(matrix, axis=1))
            matrix = self._rebalance(matrix, best_row)
            portfolio.append(best_row)
            matrix[portfolio] = np.inf

        selected_models_by_ts = np.argmin(self._matrix[portfolio], axis=0).tolist()
        return portfolio, selected_models_by_ts

    def score(
        self,
        portfolio: tp.List[int]
        ) -> float:
        return np.min(self._matrix[portfolio], axis=0).mean()

In [ ]:
def PortfolioConstriction(alghoritm,
                          horizons,
                          metric = METRIC,
                          numbers = None,
                          plot = True,
                          dataset = None):
    portfolio_dict = {}
    pypline_dict = {}
    train, test = GetDataFrameResDataValidata(dataset=dataset)

    for horizon in horizons:
        portfolio_dict[horizon] = {'models': [],
                                   'score': [],
                                   'number': [],
                                   'model_by_ts': [],
                                  }
        pypline_dict[horizon] = {'models': [],
                                 'number': [],
                                 'model_by_ts': [],
                                }

        matrix_frame = GetPortfolioMatrix(train, horizon, metric=metric)
        
        # oracle - ideal portfolio
        oracle = matrix_frame.min(axis=0).mean()
        matrix = matrix_frame.values
    
        numbers = range(1, matrix.shape[0] + 1, 1) if numbers is None else numbers
        for number in numbers:
            model = alghoritm(number=number)
            portfolio, selected_models_by_ts = model.predict(matrix)
            score = model.score(portfolio)

            portfolio_dict[horizon]['models'].append(matrix_frame.iloc[portfolio].reset_index()['model_name'].values.tolist())
            portfolio_dict[horizon]['score'].append(score)
            portfolio_dict[horizon]['number'].append(number)
            portfolio_dict[horizon]['model_by_ts'].append(selected_models_by_ts)
            
            pypline_portfolio = [np.random.choice(matrix_frame.shape[0], number, replace=False).tolist() for i in range(100)]
            selected_models_by_ts_pypline = [np.argmin(matrix_frame.iloc[portfolio].values, axis=0).tolist() for portfolio in pypline_portfolio]
            pypline_dict[horizon]['models'].append([matrix_frame.iloc[portfolio].reset_index()['model_name'].values.tolist() for portfolio in pypline_portfolio])
            pypline_dict[horizon]['number'].append(number)
            pypline_dict[horizon]['model_by_ts'].append(selected_models_by_ts_pypline)


        if plot: 
            x = np.array(portfolio_dict[horizon]['number'], dtype='int')
            plt.figure(figsize=(12, 5), layout='constrained')
            plt.xticks(x)
            plt.plot(x, np.array(portfolio_dict[horizon]['score']), label='portfolio_score')
            plt.xlabel('Number of models in the portfolio')
            plt.ylabel(f'{metric} score by portfolio')
            plt.hlines(oracle,
                    xmin=1,
                    xmax=x[-1],
                    colors='r',
                    label='oracle')
            plt.title(f'The Score ({metric}) by the number of models in portfolio on the Validation. The horizon {horizon}')
            plt.legend()
            plt.show()
    return portfolio_dict, pypline_dict

In [ ]:
def GetTestPortfolio(dict_portfolio,
                     pypeline = None,
                     metric = METRIC,
                     dataset = None,
                     plot_pypline = True,
                     plot = False):
    horizons = [int(key) for key in dict_portfolio.keys()]
    
    train, test = GetDataFrameResDataValidata(dataset=dataset)

    portfolio_dict = {}
    for horizon in horizons:
        portfolio_dict[horizon] = {'models': [],
                                   'score': [],
                                   'number': [],
                                  }
    matrix_frame = GetPortfolioMatrix(test, horizon, metric)

    score = []
    score_pypline_mean = []
    score_pypline_std = []

    for idx, number in zip(range(len(dict_portfolio[horizon]['number'])), dict_portfolio[horizon]['number']):

        selected_models = matrix_frame.loc[dict_portfolio[horizon]['models'][idx]].values
        ind = np.array(dict_portfolio[horizon]['model_by_ts'][idx]).reshape(1, -1)
        currency_score = np.take_along_axis(selected_models, ind, axis=0).mean()
        score.append(currency_score)

        if pypeline is None:
            pypeline = np.array([matrix_frame.iloc[np.random.choice(matrix_frame.shape[0],
                                                                    number,
                                                                    replace=False)].min(axis=0).mean() for i in range(100)])
            score_pypline_mean.append(pypeline.mean())
            score_pypline_std.append(pypeline.std())
        else:
            currency_score_pypline = []
            for index, portfolio_pypline in enumerate(pypeline[horizon]['models'][idx]):
                selected_models_pypline = matrix_frame.loc[portfolio_pypline].values
                ind_pypline = np.array(pypeline[horizon]['model_by_ts'][idx][index]).reshape(1, -1)
                pypline_index_score = np.take_along_axis(selected_models_pypline, ind_pypline, axis=0).mean()
                currency_score_pypline.append(pypline_index_score)
            score_pypline_mean.append(np.array(currency_score_pypline).mean())
            score_pypline_std.append(np.array(currency_score_pypline).std())

        portfolio_dict[horizon]['models'].append(dict_portfolio[horizon]['models'][idx])
        portfolio_dict[horizon]['score'].append(currency_score)
        portfolio_dict[horizon]['number'].append(number)

    oracle = matrix_frame.min(axis=0).mean()
    best_model = matrix_frame.iloc[np.argmin(np.mean(matrix_frame.values, axis=1))].mean()

    score_pypline_mean = np.array(score_pypline_mean)
    score_pypline_std = np.array(score_pypline_std)
    if plot:
        x = np.array(dict_portfolio[horizon]['number'], dtype='int')
        plt.figure(figsize=(12, 5), layout='constrained')
        plt.xticks(x)
        plt.plot(x, np.array(score), label='portfolio_score')
        plt.xlabel('Number of models in the portfolio')
        plt.ylabel(f'{metric} score by portfolio')
        plt.hlines(oracle,
                  xmin=1,
                  xmax=x[-1],
                  colors='r',
                  label=f'oracle, score = ' + '{:.3f}'.format(oracle))
        plt.hlines(best_model,
              xmin=1,
              xmax=x[-1],
              colors='y',
              label=f'best model on the test, score = ' + '{:.3f}'.format(best_model))
        
        prophet_mean = test.query(f"model_name == 'Prophet'").mean(numeric_only=True).loc[metric]
        plt.hlines(np.ones_like(best_model) * prophet_mean,
              xmin=1,
              xmax=x[-1],
              colors='r',
              label=f'Prophet test, score = ' + '{:.3f}'.format(prophet_mean))
        if plot_pypline:
            plt.errorbar(x,
                      score_pypline_mean,
                      yerr=np.minimum(score_pypline_std, 1),
                      label='random_choice_portfolio_mean_and_std',
                      ecolor='g',)
        plt.title(f'The Score ({metric}) by the number of models in portfolio on the test. The horizon {horizon}')
        plt.legend()
    return portfolio_dict

In [ ]:
from itertools import combinations

In [ ]:
result = set()

for number in range(0, len(DATASETS)):
    for dataset in combinations(DATASETS, number + 1):
        print(dataset)
        greedy_portfolio_min_smape, pypeline_min_smape = PortfolioConstriction(GreedyPortfolioConstructionMin,
                                                            [30],
                                                            numbers=list(range(1, 10)),
                                                            metric=METRIC,
                                                            dataset=dataset,
                                                            plot=False,
                                                            )
        greedy_portfolio_min_test = GetTestPortfolio(greedy_portfolio_min_smape,
                                             pypeline=pypeline_min_smape,
                                             metric=METRIC,
                                             dataset=dataset,
                                             plot_pypline=False
                                             )
        best_combination = np.array(greedy_portfolio_min_test[30]['score']).argmin()
        print(greedy_portfolio_min_test[30]['models'][best_combination])
        result.update(greedy_portfolio_min_test[30]['models'][best_combination])

In [ ]:
result

In [ ]:
greedy_portfolio_min_smape, pypeline_min_smape = PortfolioConstriction(GreedyPortfolioConstructionMin,
                                                            [30],
                                                            numbers=list(range(1, 10)),
                                                            metric=METRIC,
                                                            dataset=DATASETS,
                                                            plot=True,
                                                            )